## First steps

In [1]:
# Import Libraries
import os.path
import numpy as np
from tensorflow.python.platform import gfile
import pypcd
import os
import subprocess
import pandas as pd
from pandas import DataFrame

In [2]:
# Find number of PointClouds
point_dir = '/home/johan/Documents/Alignment/Points_csv/'
#se crea un vector vacio para guardar los nombres de las imágenes.
file_list = []
#se obtienen todos los nombre de las imagenes en la carpeta train que tengan extensión .csv
file_glob = os.path.join(point_dir, '*.csv')
#se organizan en el vector file_list todos las direcciones de las imágenes 
#encontrados dentro de la carpeta train
file_list.extend(gfile.Glob(file_glob))
file_list = np.sort(file_list)
#kind : {‘quicksort’, ‘mergesort’, ‘C’, ‘stable’}
print(len(file_list))
print(type(file_list[0]),file_list[0:8])

456
<class 'numpy.str_'> ['/home/johan/Documents/Alignment/Points_csv/pointscsv00001.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00002.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00004.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00007.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00010.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00013.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00016.csv'
 '/home/johan/Documents/Alignment/Points_csv/pointscsv00025.csv']


In [10]:
for point_dir in file_list[0:10]:
    print(point_dir)
print('\n')
for point_dir in file_list[0:5:2]:
    print(point_dir)

/home/johan/Documents/Alignment/Points_csv/pointscsv00001.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00002.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00004.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00007.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00010.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00013.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00016.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00025.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00034.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00037.csv


/home/johan/Documents/Alignment/Points_csv/pointscsv00001.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00004.csv
/home/johan/Documents/Alignment/Points_csv/pointscsv00010.csv


In [ ]:
# Alignt PCs
#para cada pointcloud dentro del vector file_list
a = 0 
refetotal = []
for point_dir in file_list:
    if a == 0:
        refe = pd.read_csv(point_dir)
        df = DataFrame(refe, columns= ['x', 'y','z'])
        export_csv = df.to_csv ('/home/johan/Documents/Alignment/Points_csv/refe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
        #print ('a0:\n refe',df.shape)
        #print('ICP finished succesfully')
        #print('------------------------------------------------------')
        
    else:
        pointcloud1_path_refe = "/home/johan/Documents/Alignment/Points_csv/refe.csv"
        refe = pd.read_csv(pointcloud1_path_refe)
        refe = np.array(refe)
        #print ("refe", refe.shape) 
        
        pointcloud2_path_in = point_dir
        icp_path="/home/johan/Libraries/libpointmatcher/build/examples"

        #--------------- correr el ejecutable de c++ ------------------
        #runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path_refe,pointcloud2_path_in,'OutPc'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
        runicp.wait()
        #-------------leer los datos de devuelve en el terminal------
        #stdout, stderr = runicp.communicate()
        
        #------------------------------------------------------------
        refe2 = pd.read_csv('/home/johan/repos/GitHub/3D-Reconstruction/refe_OutPc.csv')
        refetotal = DataFrame(refe2, columns= ['x','y','z'])
        refetotal = np.array(refetotal)
        #print('refe_OutPc',refetotal.shape, type(refetotal))
        refetotal = refetotal[1:refetotal.shape[0],:]
        #print('refe_OutPc-WOut[x,y,z]',refetotal.shape)
        refetotal_l = np.vstack((refe,refetotal))
        refetotal = DataFrame(refetotal_l) 
        #print('Acumulado',refetotal.shape)
        export_csv = refetotal.to_csv ('/home/johan/Documents/Alignment/Points_csv/refe.csv', index = None, header=True) 
        #print('ICP finished succesfully')
        #print('------------------------------------------------------') 
    
    a = a +1
    print('a:',a)

## Running ICP

In [1]:
import numpy as np
#from open3d import *
import pypcd
import os
import subprocess

In [5]:
if __name__ == '__main__':

    #------- rutas hacia las nubes de puntos y el ejecutable----------
    # .cpp file: addres /home/johan/Libraries/libpointmatcher/examples
    icp_path="/home/johan/Libraries/libpointmatcher/build/examples"
    pointcloud1_path="/home/johan/repos/GitHub/3D-Reconstruction/Dataset_csv/pointscsv1.csv"
    pointcloud2_path="/home/johan/repos/GitHub/3D-Reconstruction/Dataset_csv/pointscsv137.csv"

    #--------------- correr el ejecutable de c++ ------------------
    #runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    runicp = subprocess.Popen([os.path.join(icp_path,"icp_simple"),pointcloud1_path,pointcloud2_path,'Out'],stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    runicp.wait()
    
    stdout, stderr = runicp.communicate()
    print(stdout,'\n')
    
    # bytes to str, alternative (python3)
    #a = np.matrix(str(stdout, encoding='ascii'))
    
    a = str((stdout.decode("utf-8")))
    a = np.matrix(a)
    
    a = a.reshape(4,4)
    print(a, a.shape)

('   0.996618   0.0821525 -0.00170916     0.97926\n -0.0821504    0.996619  0.00122216 -0.00706542\n 0.00180378 -0.00107762    0.999998  0.00148535\n          0           0           0           1\n', '\n')
(matrix([[ 0.996618  ,  0.0821525 , -0.00170916,  0.97926   ],
        [-0.0821504 ,  0.996619  ,  0.00122216, -0.00706542],
        [ 0.00180378, -0.00107762,  0.999998  ,  0.00148535],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]), (4, 4))


## Final Code ICP-ROS

In [2]:
import rospy
import message_filters
from sensor_msgs.msg import Image, PointCloud2
import sys
from cv_bridge import CvBridge, CvBridgeError
#import cv2
import numpy as np
import laser_geometry.laser_geometry as lg
import sensor_msgs.point_cloud2 as pc2
import open3d as opn3

In [ ]:
class synchronizer:
	def __init__(self):
		#self.laserProj = LaserProjection()
		self.bridge = CvBridge()
		#self.image_sub = message_filters.Subscriber('/left/image_rect_color', Image)
        self.pc_pub = message_filters.Publisher('/Dense',PointCloud2,queue_size=2)
		self.pc_sub = message_filters.Subscriber('/kitti/velo/pointcloud', PointCloud2,self.Callback)
		
	def callback(self, pointcloud):
         cloud_pub.publish(pointcloud)
        
		
def main(args):
  print('Starting...')
  rospy.init_node('sync_node', anonymous=True)
  rospy.loginfo("sync_node on")
  #sc = synchronizer()
  
  try:
    rospy.spin()
  except KeyboardInterrupt:
    print("Shutting down")

if __name__ == '__main__':
	
    main(sys.argv)

## OPen3D

In [13]:
import open3d as o3d
import numpy as np
import copy

In [31]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.draw_geometries([source_temp, target_temp])



source = o3d.read_point_cloud("/home/johan/Documents/Alignment/INFo_Bag8/Points/points00001.pcd")
target = o3d.read_point_cloud("/home/johan/Documents/Alignment/INFo_Bag8/Points/points00023.pcd")

threshold = 0.3
trans_init = np.asarray(
            [[1, 0, 0,  0],
            [0, 1, 0,  0],
            [0, 0,  1, 0],
            [0, 0, 0, 1]])


print("Apply point-to-plane ICP")
reg_p2l = o3d.registration_icp(source, target, threshold, trans_init,
o3d.TransformationEstimationPointToPoint())
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
print("")
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness = 0.744423, inlier_rmse = 0.102642, and correspondence_set size of 20657
Access transformation to get result.
Transformation is:
[[ 9.97600647e-01  6.92309670e-02 -1.47540839e-04  6.43771233e-03]
 [-6.92310697e-02  9.97594924e-01 -3.38043733e-03 -2.97221163e-02]
 [-8.68449533e-05  3.38254088e-03  9.99994275e-01 -2.39720663e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



**It is very important to initialized the trans_ini in order to have a good ICP performance.**

In [28]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.draw_geometries([source_temp, target_temp])

source = o3d.read_point_cloud("/home/johan/Documents/Alignment/INFo_Bag8/Points/points00001.pcd")
target = o3d.read_point_cloud("/home/johan/Documents/Alignment/INFo_Bag8/Points/points00023.pcd")

threshold = 0.9

trans_init = np.asarray([[ 9.97600647e-01,  6.92309670e-02, -1.47540839e-04,  6.43771233e-03],
 [-6.92310697e-02,  9.97594924e-01 ,-3.38043733e-03, -2.97221163e-02],
 [-8.68449533e-05 , 3.38254088e-03 , 9.99994275e-01 ,-2.39720663e-02],
 [ 0.00000000e+00,  0.00000000e+00 , 0.00000000e+00 , 1.00000000e+00]])

print("Apply point-to-plane ICP")
reg_p2l = o3d.registration_icp(source, target, threshold, trans_init,
o3d.TransformationEstimationPointToPoint())
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
print("")
draw_registration_result(source, target, reg_p2l.transformation)

Apply point-to-plane ICP
RegistrationResult with fitness = 0.980071, inlier_rmse = 0.178639, and correspondence_set size of 27196
Access transformation to get result.
Transformation is:
[[ 9.90654806e-01  1.36366564e-01  2.68617487e-03 -3.17922693e-01]
 [-1.36369004e-01  9.90657831e-01  7.46351727e-04 -8.69739206e-03]
 [-2.55930275e-03 -1.10568792e-03  9.99996113e-01 -6.07230188e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



**threshold = 0.3**   
Apply point-to-plane ICP   
RegistrationResult with fitness = 0.903672, inlier_rmse = 0.092669, and correspondence_set size of 25076   
Access transformation to get result.  

### Testing OPen 3D using the bag8

In [44]:
source = o3d.read_point_cloud("/home/johan/Documents/Alignment/INFo_Bag8/Points/points00001.pcd")
target = o3d.read_point_cloud("/home/johan/Documents/Alignment/INFo_Bag8/Points/points00002.pcd")
# convert Open3D.o3d.geometry.PointCloud to numpy array
target_ICP = np.asarray(target.points)
print('target_ICP', target_ICP.shape)
colum = np.ones((target_ICP.shape[0],1))
target_ICP = np.hstack((target_ICP,colum))
print('target_ICP', target_ICP.shape)

threshold = 0.3
trans_init = np.asarray(
            [[1, 0, 0,  0],
            [0, 1, 0,  0],
            [0, 0,  1, 0],
            [0, 0, 0, 1]])


print("Apply point-to-plane ICP")
reg_p2l = o3d.registration_icp(source, target, threshold, trans_init,
o3d.TransformationEstimationPointToPoint())
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
print("")

target_ICP_DOT = np.dot(target_ICP,reg_p2l.transformation)
target_ICP_DOT = target_ICP_DOT[:,0:3]
pcd = o3d.PointCloud()
pcd.points = o3d.Vector3dVector(target_ICP_DOT)
o3d.write_point_cloud("/home/johan/Documents/Alignment/INFo_Bag8/PCD_Transformations_Open3D/02ICP.pcd", pcd)

#draw_registration_result(source, target, reg_p2l.transformation)

('target_ICP', (27702, 3))
('target_ICP', (27702, 4))
Apply point-to-plane ICP
RegistrationResult with fitness = 0.949800, inlier_rmse = 0.068289, and correspondence_set size of 26356
Access transformation to get result.
Transformation is:
[[ 9.99821864e-01  1.88741586e-02  7.46947191e-05 -2.45684327e-02]
 [-1.88741556e-02  9.99821866e-01 -4.09616643e-05  6.28392027e-03]
 [-7.54545304e-05  3.95445678e-05  9.99999996e-01 -2.42031131e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]



True